# Convolutional Neural Network

### Importing the libraries

In [78]:
import tensorflow as tf
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator


In [79]:
tf.__version__

'2.9.1'

## Data Preprocessing

### Preprocessing the Training set

In [80]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_directory('input/Training',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
labels =training_set.class_indices
print(labels)


Found 5712 images belonging to 4 classes.
{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}


### Preprocessing the Test set

In [81]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('input/Testing',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1311 images belonging to 4 classes.


## Building the CNN

### Initialising the CNN

In [82]:
cnn = tf.keras.models.Sequential()

### Convolution

In [83]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[224, 224, 3]))

### Pooling

In [84]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [85]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Adding a third convolutional layer

In [86]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Flattening

In [87]:
cnn.add(tf.keras.layers.Flatten())

### Full Connection

In [88]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=32, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.3))

### Output Layer

In [89]:
cnn.add(tf.keras.layers.Dense(units=4, activation='softmax'))

## Training the CNN

### Compiling the CNN

In [90]:
cnn.compile(optimizer = Adam(learning_rate=0.003), loss = 'categorical_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [91]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25, verbose = 1 , callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)])

Epoch 1/25
179/179 [==============================] - 220s 1s/step - loss: 0.9923 - accuracy: 0.5807 - val_loss: 0.7018 - val_accuracy: 0.7300
Epoch 2/25
179/179 [==============================] - 213s 1s/step - loss: 0.6640 - accuracy: 0.7402 - val_loss: 0.7136 - val_accuracy: 0.7422
Epoch 3/25
179/179 [==============================] - 210s 1s/step - loss: 0.5565 - accuracy: 0.7890 - val_loss: 0.5651 - val_accuracy: 0.7597
Epoch 4/25
179/179 [==============================] - 197s 1s/step - loss: 0.4724 - accuracy: 0.8295 - val_loss: 0.4706 - val_accuracy: 0.8047
Epoch 5/25
179/179 [==============================] - 151s 844ms/step - loss: 0.3981 - accuracy: 0.8508 - val_loss: 0.4924 - val_accuracy: 0.7948
Epoch 6/25
179/179 [==============================] - 149s 829ms/step - loss: 0.3887 - accuracy: 0.8561 - val_loss: 0.4558 - val_accuracy: 0.8299
Epoch 7/25
179/179 [==============================] - 159s 886ms/step - loss: 0.3375 - accuracy: 0.8771 - val_loss: 0.3165 - val_accurac

## Making a single prediction

In [95]:
import numpy as np
from keras import utils
test_image = utils.load_img('input/Testing/notumor/Te-no_0018.jpg', target_size = (224, 224))
test_image = utils.img_to_array(test_image) /255.0
test_image = np.expand_dims(test_image, axis = 0) # Convert single image to a batch.
predicted_indice_arr = cnn.predict(test_image)
print(predicted_indice_arr)
prediction = np.argmax(predicted_indice_arr, axis=1)

print(labels)
final_prediction = [key for key, val in labels.items() if val == prediction[0]]
print(final_prediction)


1/1 [==============================] - 0s 111ms/step
[[4.9810572e-10 1.9925607e-04 9.9980074e-01 7.5695986e-12]]
{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}
['notumor']


In [99]:
test_image = utils.load_img('input/Testing/pituitary/Te-pi_0056.jpg', target_size = (224, 224))
test_image = utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0) # Convert single image to a batch.
predicted_indice_arr = cnn.predict(test_image) /255.0
print(predicted_indice_arr)
prediction = np.argmax(predicted_indice_arr, axis=1)

print(labels)
final_prediction = [key for key, val in labels.items() if val == prediction[0]]
print(final_prediction)


1/1 [==============================] - 0s 34ms/step
[[0.0000000e+00 3.9215689e-03 1.8020093e-21 0.0000000e+00]]
{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}
['meningioma']


In [98]:
test_image = utils.load_img('input/Testing/glioma/Te-gl_0078.jpg', target_size = (224, 224))
test_image = utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0) # Convert single image to a batch.
predicted_indice_arr = cnn.predict(test_image) /255.0
print(predicted_indice_arr)
prediction = np.argmax(predicted_indice_arr, axis=1)

print(labels)
final_prediction = [key for key, val in labels.items() if val == prediction[0]]
print(final_prediction)


1/1 [==============================] - 0s 34ms/step
[[0.         0.         0.00392157 0.        ]]
{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}
['notumor']


In [97]:
test_image = utils.load_img('input/Testing/meningioma/Te-me_0067.jpg', target_size = (224, 224))
test_image = utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0) # Convert single image to a batch.
predicted_indice_arr = cnn.predict(test_image) /255.0
print(predicted_indice_arr)
prediction = np.argmax(predicted_indice_arr, axis=1)

print(labels)
final_prediction = [key for key, val in labels.items() if val == prediction[0]]
print(final_prediction)


1/1 [==============================] - 0s 49ms/step
[[0.0000000e+00 3.9215689e-03 1.9855457e-23 0.0000000e+00]]
{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}
['meningioma']
